In [1]:
import os
import math
import copy
import collections
import networkx as nx

In [2]:
def aoc_2021_23_1(file_path):
    """--- Day 23: Amphipod --- Part One"""
    
    def move_amphibod(loc_dict, energy):
        """Dynamic programming to find minimal energy of amphibod movements"""

        if not [a for a in loc_dict if not a.endswith('_final')]:
            return 0

        sorted_loc = tuple([loc_dict[a] for a in sorted(loc_dict)])
        if sorted_loc in min_energy_dict:
            return min_energy_dict[sorted_loc]

        occupied_spots = set(loc_dict.values())
        new_movements = []

        # calculate room final position of each amphibod
        final_count = collections.Counter([a[0] for a in loc_dict if a.endswith('_final')])
        final_room_dict = {}
        for letter in letter_list:
            if letter not in final_count:
                final_room_dict[letter] = letter_list.index(letter) + 11
            elif final_count[letter] < len(loc_dict) / 4:
                final_room_dict[letter] = final_count[letter] * 10 + letter_list.index(letter) + 11

        # for each amphibod to move
        for amphibod, loc in loc_dict.items():
            if amphibod.endswith('_final'):
                continue

            # occupied spots
            temp_occupied_spots = occupied_spots - {loc}

            # leave room
            if loc < no_rows * 10:

                # if next move can return amphibod directly            
                temp_return = list(nx.shortest_path(amphibod_graph, loc, final_room_dict[amphibod[0]]))
                if not set(temp_return).intersection(temp_occupied_spots):
                    movement = temp_return

                    new_loc_dict = copy.deepcopy(loc_dict)
                    new_loc_dict['%s_final' % amphibod] = movement[-1]
                    del new_loc_dict[amphibod]

                    temp_energy = amphibod_score[amphibod[0]] * sum([amphibod_graph.edges[(movement[i_m], movement[i_m + 1])]['distance'] for i_m in range(len(movement) - 1)])
                    new_movements.append((new_loc_dict, temp_energy))
                    continue

                # move to top row
                temp_paths = [list(nx.shortest_path(amphibod_graph, loc, t)) for t in top_row - temp_occupied_spots]
                temp_paths = [p for p in temp_paths if not set(p).intersection(temp_occupied_spots)]

                # next move
                for movement in temp_paths:
                    new_loc_dict = copy.deepcopy(loc_dict)
                    new_loc_dict[amphibod] = movement[-1]

                    temp_energy = amphibod_score[amphibod[0]] * sum([amphibod_graph.edges[(movement[i_m], movement[i_m + 1])]['distance'] for i_m in range(len(movement) - 1)])
                    new_movements.append((new_loc_dict, temp_energy))

            # return to a room
            else:  

                # if next move can return amphibod directly            
                temp_return = list(nx.shortest_path(amphibod_graph, loc, final_room_dict[amphibod[0]]))
                if not set(temp_return).intersection(temp_occupied_spots):
                    movement = temp_return

                    new_loc_dict = copy.deepcopy(loc_dict)
                    new_loc_dict['%s_final' % amphibod] = movement[-1]
                    del new_loc_dict[amphibod]

                    temp_energy = amphibod_score[amphibod[0]] * sum([amphibod_graph.edges[(movement[i_m], movement[i_m + 1])]['distance'] for i_m in range(len(movement) - 1)])
                    new_movements.append((new_loc_dict, temp_energy))

        # memoize minimal score
        if not new_movements:
            min_energy_dict[sorted_loc] = math.inf
            return math.inf
        else:
            minimal_energy_list = []
            for movement in new_movements:
                minimal_energy_list.append(move_amphibod(movement[0], movement[1]))

            current_min_score = min([new_movements[i_m][1] + minimal_energy_list[i_m] for i_m in range(len(new_movements))])
            min_energy_dict[sorted_loc] = current_min_score
            return current_min_score
    
    # main function
    with open(file_path) as f:
        aoc_read = f.read().split('\n')

   # setup amphibod map
    amphibod_graph = nx.Graph()
    no_rows = 3

    # rooms
    edge_distance_dict = {}
    for row in range(1, no_rows - 1):
        for column in range(1, 5):
            edge_distance_dict[(row * 10 + column), (row * 10 + 10 + column)] = 1

    # rooms to hallways
    for column in range(1, 5):
        edge_distance_dict[((no_rows - 1) * 10 + column, no_rows * 10 + column)] = 2
        edge_distance_dict[((no_rows - 1) * 10 + column, no_rows * 10 + column + 1)] = 2

    # hallways
    for column in range(1, 5):
        edge_distance_dict[(no_rows * 10 + column, no_rows * 10 + column + 1)] = 2
    edge_distance_dict[(no_rows * 10, no_rows * 10 + 1)] = 1
    edge_distance_dict[(no_rows * 10 + 5, no_rows * 10 + 6)] = 1

    # setup graph
    amphibod_graph.add_edges_from(sorted(edge_distance_dict))
    top_row = set([n for n in amphibod_graph.nodes if n >= no_rows * 10])

    # distance between each location
    for edge, distance in edge_distance_dict.items():
        amphibod_graph.edges[edge]['distance'] = distance
    all_loc = set(amphibod_graph.nodes)

    # amphibod rules
    letter_list = ['A', 'B', 'C', 'D']
    amphibod_score = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

    # initialize amphibod location
    amphibod_list = [aoc_read[3][i] for i in [3, 5, 7, 9]] + [aoc_read[2][i] for i in [3, 5, 7, 9]]
    for replace in letter_list:
        amphibod_list[amphibod_list.index(replace)] = '%s1' % replace
        amphibod_list[amphibod_list.index(replace)] = '%s2' % replace
    initial_amphibod_loc = [11, 12, 13, 14, 21, 22, 23, 24]
    initial_loc_dict = {}
    for i_amphibod, amphibod in enumerate(amphibod_list):
        temp_loc = initial_amphibod_loc[i_amphibod]
        if temp_loc == letter_list.index(amphibod[0]) + 11:
            initial_loc_dict['%s_final' % amphibod] = initial_amphibod_loc[i_amphibod]
        else:
            initial_loc_dict[amphibod] = initial_amphibod_loc[i_amphibod]
            
    min_energy_dict = {}
    min_energy = move_amphibod(initial_loc_dict, 0)
    return min_energy

In [3]:
aoc_2021_23_1('example.txt')

12521

In [4]:
aoc_2021_23_1('input.txt')

16300

In [5]:
def aoc_2021_23_2(file_path):
    """--- Day 23: Amphipod --- Part Two"""
    
    def move_amphibod(loc_dict, energy):
        """Dynamic programming to find minimal energy of amphibod movements"""

        if not [a for a in loc_dict if not a.endswith('_final')]:
            return 0

        sorted_loc = tuple([loc_dict[a] for a in sorted(loc_dict)])
        if sorted_loc in min_energy_dict:
            return min_energy_dict[sorted_loc]

        occupied_spots = set(loc_dict.values())
        new_movements = []

        # calculate room final position of each amphibod
        final_count = collections.Counter([a[0] for a in loc_dict if a.endswith('_final')])
        final_room_dict = {}
        for letter in letter_list:
            if letter not in final_count:
                final_room_dict[letter] = letter_list.index(letter) + 11
            elif final_count[letter] < len(loc_dict) / 4:
                final_room_dict[letter] = final_count[letter] * 10 + letter_list.index(letter) + 11

        # for each amphibod to move
        for amphibod, loc in loc_dict.items():
            if amphibod.endswith('_final'):
                continue

            # occupied spots
            temp_occupied_spots = occupied_spots - {loc}

            # leave room
            if loc < no_rows * 10:

                # if next move can return amphibod directly            
                temp_return = list(nx.shortest_path(amphibod_graph, loc, final_room_dict[amphibod[0]]))
                if not set(temp_return).intersection(temp_occupied_spots):
                    movement = temp_return

                    new_loc_dict = copy.deepcopy(loc_dict)
                    new_loc_dict['%s_final' % amphibod] = movement[-1]
                    del new_loc_dict[amphibod]

                    temp_energy = amphibod_score[amphibod[0]] * sum([amphibod_graph.edges[(movement[i_m], movement[i_m + 1])]['distance'] for i_m in range(len(movement) - 1)])
                    new_movements.append((new_loc_dict, temp_energy))
                    continue

                # move to top row
                temp_paths = [list(nx.shortest_path(amphibod_graph, loc, t)) for t in top_row - temp_occupied_spots]
                temp_paths = [p for p in temp_paths if not set(p).intersection(temp_occupied_spots)]

                # next move
                for movement in temp_paths:
                    new_loc_dict = copy.deepcopy(loc_dict)
                    new_loc_dict[amphibod] = movement[-1]

                    temp_energy = amphibod_score[amphibod[0]] * sum([amphibod_graph.edges[(movement[i_m], movement[i_m + 1])]['distance'] for i_m in range(len(movement) - 1)])
                    new_movements.append((new_loc_dict, temp_energy))

            # return to a room
            else:  

                # if next move can return amphibod directly            
                temp_return = list(nx.shortest_path(amphibod_graph, loc, final_room_dict[amphibod[0]]))
                if not set(temp_return).intersection(temp_occupied_spots):
                    movement = temp_return

                    new_loc_dict = copy.deepcopy(loc_dict)
                    new_loc_dict['%s_final' % amphibod] = movement[-1]
                    del new_loc_dict[amphibod]

                    temp_energy = amphibod_score[amphibod[0]] * sum([amphibod_graph.edges[(movement[i_m], movement[i_m + 1])]['distance'] for i_m in range(len(movement) - 1)])
                    new_movements.append((new_loc_dict, temp_energy))

        # memoize minimal score
        if not new_movements:
            min_energy_dict[sorted_loc] = math.inf
            return math.inf
        else:
            minimal_energy_list = []
            for movement in new_movements:
                minimal_energy_list.append(move_amphibod(movement[0], movement[1]))

            current_min_score = min([new_movements[i_m][1] + minimal_energy_list[i_m] for i_m in range(len(new_movements))])
            min_energy_dict[sorted_loc] = current_min_score
            return current_min_score
    
    # main function
    with open(file_path) as f:
        aoc_read = f.read().split('\n')

    # setup amphibod map
    amphibod_graph = nx.Graph()
    no_rows = 5

    # rooms
    edge_distance_dict = {}
    for row in range(1, no_rows - 1):
        for column in range(1, 5):
            edge_distance_dict[(row * 10 + column), (row * 10 + 10 + column)] = 1

    # rooms to hallways
    for column in range(1, 5):
        edge_distance_dict[((no_rows - 1) * 10 + column, no_rows * 10 + column)] = 2
        edge_distance_dict[((no_rows - 1) * 10 + column, no_rows * 10 + column + 1)] = 2

    # hallways
    for column in range(1, 5):
        edge_distance_dict[(no_rows * 10 + column, no_rows * 10 + column + 1)] = 2
    edge_distance_dict[(no_rows * 10, no_rows * 10 + 1)] = 1
    edge_distance_dict[(no_rows * 10 + 5, no_rows * 10 + 6)] = 1

    # setup graph
    amphibod_graph.add_edges_from(sorted(edge_distance_dict))
    top_row = set([n for n in amphibod_graph.nodes if n >= no_rows * 10])

    # distance between each location
    for edge, distance in edge_distance_dict.items():
        amphibod_graph.edges[edge]['distance'] = distance
    all_loc = set(amphibod_graph.nodes)

    # amphibod rules
    letter_list = ['A', 'B', 'C', 'D']
    amphibod_score = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

    # initialize amphibod location
    amphibod_list = [aoc_read[3][i] for i in [3, 5, 7, 9]] + ['D', 'B', 'A', 'C', 'D', 'C', 'B', 'A'] + [aoc_read[2][i] for i in [3, 5, 7, 9]]
    for replace in letter_list:
        amphibod_list[amphibod_list.index(replace)] = '%s1' % replace
        amphibod_list[amphibod_list.index(replace)] = '%s2' % replace
        amphibod_list[amphibod_list.index(replace)] = '%s3' % replace
        amphibod_list[amphibod_list.index(replace)] = '%s4' % replace
    initial_amphibod_loc = [11, 12, 13, 14, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]
    initial_loc_dict = {}
    for i_amphibod, amphibod in enumerate(amphibod_list):
        temp_loc = initial_amphibod_loc[i_amphibod]
        if temp_loc == letter_list.index(amphibod[0]) + 11:
            initial_loc_dict['%s_final' % amphibod] = initial_amphibod_loc[i_amphibod]
        else:
            initial_loc_dict[amphibod] = initial_amphibod_loc[i_amphibod]
            
    min_energy_dict = {}
    min_energy = move_amphibod(initial_loc_dict, 0)
    return min_energy

In [6]:
aoc_2021_23_2('example.txt')

44169

In [7]:
aoc_2021_23_2('input.txt')

48676